# Descriptif
Ce programme a pour but de vérifier que les solutions que vous générez respectent bien les contraintes du problème.
Il permet également de calculer la performance obtenue.
La pénalisation des ventes manquées est possible, il faut en préciser le coût, qui est défini comme un pourcentage du prix de vente du produit non vendu.

Il s'agit d'une première version, pour l'instant la production dans les plants n'est pas gérée, le stock initial dans les plants est de 500 000 unités ce qui permet de ne jamais être en rupture.

In [ ]:
Fichier_json='solutions_3jours.json'
Fichier_vente='sales.csv'
Vente_manquee=0

In [2]:
#créer des structures enregistrant les niveaux de stocks
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
Plant=numpy.zeros(shape=(3,2)) # premier indice numero de warehous, 2ieme indices 0 heater 1 clim
Plant[1]=[500000, 500000]
Plant[2]=[500000, 500000]

#stocks initiaux
Warehouse=numpy.zeros(shape=(3,2)) # premier indice numero de warehous, 2ieme indices 0 heater 1 clim
Shop=numpy.zeros(shape=(21,4))

#importer les ventes
import pandas as pd
vente_pd = pd.read_csv(Fichier_vente)

#Récupere les données du fichier Json
from pprint import pprint
import json
data = []
with open(Fichier_json, encoding='utf-8') as data_file:
    data = json.loads(data_file.read())


In [3]:
#definition des capacités
W_capa=650
S_capa=numpy.zeros(shape=(21,2))
S_capa[1]=[15,15]
S_capa[2]=[15,15]
S_capa[3]=[20,20]
S_capa[4]=[20,20]
S_capa[5]=[15,15]
S_capa[6]=[20,20]
S_capa[7]=[30,30]
S_capa[8]=[30,30]
S_capa[9]=[35,35]
S_capa[10]=[25,25]
S_capa[11]=[30,30]
S_capa[12]=[30,30]
S_capa[13]=[30,30]
S_capa[14]=[35,35]
S_capa[15]=[30,30]
S_capa[16]=[40,40]
S_capa[17]=[25,25]
S_capa[18]=[20,20]
S_capa[19]=[15,15]
S_capa[20]=[20,20]

In [4]:
##### coordonnées
P=numpy.zeros(shape=(3,2))
P[1]=[1,2]
P[2]=[6,5]
W=numpy.zeros(shape=(3,2))
W[1]=[4,3]
W[2]=[9,4]
S=numpy.zeros(shape=(21,2))
S[1]=[2,0]           
S[2]=[10,0]           
S[3]=[3,1]           
S[4]=[5,1]           
S[5]=[8,1]          
S[6]=[0,2]           
S[7]=[2,3]           
S[8]=[6,3]           
S[9]=[10,3]           
S[10]=[12,3]           
S[11]=[1,4]           
S[12]=[4,4]           
S[13]=[11,4]         
S[14]=[3,5]           
S[15]=[8,5]           
S[16]=[10,5]           
S[17]=[2,6]
S[18]=[4,6]                 
S[19]=[7,6]           
S[20]=[12,6]


#definition des distances plant_warehouse
P_W=numpy.zeros(shape=(3,3))
P_W[1]=[0, 4, 10]
P_W[2]=[0, 4 ,4]
       
#definition des distances warehouse_warehouse
W_W=numpy.zeros(shape=(3,3))
W_W[1]=[0, 0, 6]
W_W[2]=[0, 6 ,0]
       
#definition des distances warehouse_shop
W_S=numpy.zeros(shape=(3,21))
for i in range(1,3):
    for j in range(1,21):
        W_S[i][j]=abs(W[i][0]-S[j][0])+abs(W[i][1]-S[j][1])

#definition des distances shop_shop
S_S=numpy.zeros(shape=(21,21))
for i in range(1,21):
    for j in range(1,21):
        S_S[i][j]=abs(S[i][0]-S[j][0])+abs(S[i][1]-S[j][1])
       

Les blocs suivants définissent différentes fonctions qui seront appellées dans le programme principal

In [5]:
#vérification du respect des capacités des camions
def Capacite_truck_respectee(camion,j):
    capa=0
    nb_heater=0
    nb_clim=0
    for k in range(len(camion["delivery"])):#sur tous les points de livraison                
        for l in range(len(camion["delivery"][k]["products"])):#sur tous les produits                
            if (camion["delivery"][k]["products"][l]["name"] == "heater"):
                nb_heater+=camion["delivery"][k]["products"][l]["quantity"]
                capa=capa+0.4*camion["delivery"][k]["products"][l]["quantity"]
            if (camion["delivery"][k]["products"][l]["name"] == "conditioner"):
                nb_clim+=camion["delivery"][k]["products"][l]["quantity"]
                capa=capa+0.8*camion["delivery"][k]["products"][l]["quantity"]
    if (capa<=20):
        print('capacité du camion '+str(j)+' respectée: charge = '+str(capa))
        return(nb_heater,nb_clim,True)
    else:
        print('Problème : camion '+str(j)+' trop chargé!! : charge = '+str(capa))
        return(0,0,False)        
           

In [6]:
#réalisation des livraisons plant-> warehouse
def livraison_plant_warehouse(camion,nb_heater,nb_clim, Bool_affiche=False):
    local_OK=True
    numero_plant=int(camion["home"][1:])  
    #vérification de la possibilité de charger le camion selon le bon de livraison
    if nb_clim>Plant[numero_plant][1]: 
        print('Problème :pas de stock de clim dans la plant'+ str(numero_plant))
        local_OK=False
                    
    else :
        Plant[numero_plant][1]=Plant[numero_plant][1]-nb_clim
        
    if (nb_heater>Plant[numero_plant][0]):                   
        print('Problème : pas de stock de heater dans la plant '+str(numero_plant))
        local_OK=False
    else :
        Plant[numero_plant][0]=Plant[numero_plant][0]-nb_heater
                    #print('nouveau stock de heater dans la warehouse '+str(numero)+' : '+str(Warehouse[numero-1][0]))
                    #il est possible de livrer, livraison dans les shop
    if local_OK:            
        for k in range(len(camion["delivery"])):#sur tous les points de livraison                         
            
            if (camion["delivery"][k]["name"][:1]=='W'):
                numero_w=int(camion["delivery"][k]["name"][1:]) 
                print('il s arrete à la warehouse '+str(numero_w))
                                            
                for m in range(len(camion["delivery"][k]["products"])):                                 
                    if (camion["delivery"][k]["products"][m]["name"] == "heater"):                      
                        Warehouse[numero_w][0]+=camion["delivery"][k]["products"][m]["quantity"]#heater
                        if Warehouse[numero_w][0]>W_capa:
                            Plant[numero_plant][0]= Plant[numero_plant][0]-(Warehouse[numero_w][0]-W_capa)
                            Warehouse[numero_w][0]=W_capa
                    if (camion["delivery"][k]["products"][m]["name"] == "conditioner"):  
                            Warehouse[numero_w][1]+=camion["delivery"][k]["products"][m]["quantity"]#clim                                       
                            if Warehouse[numero_w][1]>W_capa:
                                Plant[numero_plant][1]= Plant[numero_plant][1]-(Warehouse[numero_w][1]-W_capa) 
                                Warehouse[numero_w][1]=W_capa                                    
            else: 
                print('Problème :le camion part d une plant mais ne va pas à une warehouse')
                local_OK=False
    if not local_OK:
        print('Probleme sur cette livraison')
    else:
          if Bool_affiche:
              Afficher(Warehouse,'Warehouses','stocks apres reception des envois des plants')
    return local_OK

In [1]:
#réalisation des livraisons  warehouse->shop
def livraison_warehouse_shop(camion,nb_heater,nb_clim,Bool_affiche=False):
    local_OK=True    
    numero_w=int(camion["home"][1:])            
   
    
    for k in range(len(camion["delivery"])):#sur tous les points de livraison                                                        
        numero_shop=int(camion["delivery"][k]["name"][1:])                       
        if (camion["delivery"][k]["name"][:1]=='S'):
            for m in range(len(camion["delivery"][k]["products"])):                                
                if (camion["delivery"][k]["products"][m]["name"] == "heater"):
                    if (camion["delivery"][k]["products"][m]["quantity"] >Warehouse[numero_w][0]): 
                        print('Problème : pas assez de stock de heater dans la warehouse '+str(numero_w))
                        Shop[numero_shop][0]+=Warehouse[numero_w][0]#heater
                        Warehouse[numero_w][0]=0  
                    else:
                        Shop[numero_shop][0]+=camion["delivery"][k]["products"][m]["quantity"]#heater
                        Warehouse[numero_w][0]=Warehouse[numero_w][0]-camion["delivery"][k]["products"][m]["quantity"]
                        if (Shop[numero_shop][0]>S_capa[numero_shop][0]):
                            Warehouse[numero_w][0]+=(Shop[numero_shop][0]-S_capa[numero_shop][0])#produit renvoyé dans la Warehouse
                            Shop[numero_shop][0]=S_capa[numero_shop][0]
                elif (camion["delivery"][k]["products"][m]["name"] == "conditioner"):
                    if camion["delivery"][k]["products"][m]["quantity"]>Warehouse[numero_w][1]: 
                        print('Problème : pas assez  de stock de clim dans la warehouse'+ str(numero_w))
                        Shop[numero_shop][1]+=Warehouse[numero_w][1]#clim
                        Warehouse[numero_w][1]=0                       
                    else:    
                        Shop[numero_shop][1]+=data[i]["trucks"][j]["delivery"][k]["products"][m]["quantity"]#clim
                        Warehouse[numero_w][1]=Warehouse[numero_w][1]-camion["delivery"][k]["products"][m]["quantity"]   
                        if (Shop[numero_shop][1]>S_capa[numero_shop][1]):
                            Warehouse[numero_w][1]+=(Shop[numero_shop][1]-S_capa[numero_shop][1])#produit renvoyé dans la Warehouse
                            Shop[numero_shop][1]=S_capa[numero_shop][1]
        else:
            print('le camion part dune warehouse mais ne va pas à un shop')
            local_OK=False
    
    if not local_OK:
        print('livraison_warehouse_shop local_ok '+str(local_OK))
    else:
        if Bool_affiche:
            Afficher(Shop,'Shops','stocks apres reception des envois des warehouse')
    return local_OK

In [8]:
#calcul du nombre de la distance parcourue
def calcul_cout(camion):
    km=0  
    #pour chaque point de livraison
    if camion["home"][:1]=="P":
        numero_p=int(camion["home"][1:])
         #distance home->name[first]
        numero_w=int(camion["delivery"][0]["name"][1:])
        distance=P_W[numero_p][numero_w]
        #print(distance)
        km+=distance   
        print('Tournée P'+str(numero_p)+'->W'+str(numero_w))        
        for k in range(len(camion["delivery"])-1): 
            numero_w1=int(camion["delivery"][k]["name"][1:])
            numero_w2=int(camion["delivery"][k+1]["name"][1:])
            print('il va de W'+str(numero_w1)+' a W'+str(numero_w2))
            distance=W_W[numero_w1][numero_w2]        
            km+=distance
        #distance name[last]->home
        numero_w=int(camion["delivery"][len(camion["delivery"])-1]["name"][1:])
        distance=P_W[numero_p][numero_w]        
        km+=distance 
        print('retour W'+str(numero_w)+'->P'+str(numero_p))   
            
    elif camion["home"][:1]=="W":
        numero_w=int(camion["home"][1:])
         #distance home->name[first]
        numero_s=int(camion["delivery"][0]["name"][1:])
        distance=W_S[numero_w][numero_s]
        #print(distance)
        km+=distance
        print('Tournée W'+str(numero_w)+'->S'+str(numero_s))      
        
                 
        for k in range(len(camion["delivery"])-1): 
            numero_s1=int(camion["delivery"][k]["name"][1:])
            numero_s2=int(camion["delivery"][k+1]["name"][1:])
            print('il va de S'+str(numero_s1)+' a S'+str(numero_s2))
            distance=S_S[numero_s1][numero_s2]        
            km+=distance
        #distance name[last]->home
        numero_s=int(camion["delivery"][len(camion["delivery"])-1]["name"][1:])
        distance=W_S[numero_w][numero_s]        
        km+=distance    
        print('retour S'+str(numero_s)+'->W'+str(numero_w))       
                #distance name[ici]->name[ici+1] if ici+1<=last
    return km

In [9]:
#Affichage de l histogramme des niveaux de stocks
def Afficher(niveau,str1,str2):
     
    N = len(niveau)
    #x = range(N)
    x=numpy.arange(N)    
    ax = plt.subplot(111)
    rects1 =ax.bar(x+0.2, niveau[:,0],width=0.2,color='b',align='center')
    rects2 =ax.bar(x+0.4, niveau[:,1],width=0.2,color='r',align='center')
    ax.legend( (rects1[0], rects2[0]), ('Heaters en stock ',  'Clim en stock') )
    plt.ylabel('Volumes')
    plt.xlabel(str1)
    plt.title(str2)
    plt.xlim((1,len(niveau)+1))
    plt.show()
    return

In [10]:
#Affichage de l histogramme des niveaux de stocks et des ventes réalisées
def Afficher_vente(niveau,str1, str2):#   Afficher_vente(Shop_loc,'Shop','Bilan de la journée')     
    N = len(niveau)
    #x = range(N)
    x=numpy.arange(N)       
    ax = plt.subplot(111)
    rects1 =ax.bar(x, niveau[:,0],width=0.2,color='b',align='center')
    rects2 =ax.bar(x+0.2, niveau[:,2],width=0.2,color='b',alpha=0.5,align='center')
    rects3 =ax.bar(x+0.4, niveau[:,1],width=0.2,color='r',align='center')
    rects4 =ax.bar(x+0.6, niveau[:,3],width=0.2,color='r',alpha=0.5,align='center')
    ax.legend( (rects1[0], rects2[0], rects3[0], rects4[0]), ('Heaters en stock matin', 'Heaters vendus', 'Clim en stocks matin', 'Clim vendues') )
    plt.ylabel('Volumes')
    plt.xlabel(str1)
    plt.title(str2)
    plt.xlim((1,len(niveau)+1))
    plt.show()
    return

In [11]:
#Routine principale
# trier les date dans l'ordre chornologique
sorted(data,key=lambda obj: obj["date"])
print([obj["date"] for obj in data])
OK=True
cout_global=0
cout_ventes_manquees=0
profit=0
    #Chaque jour
for i in range(len(data)):    
    print('jour '+ str(i)+' : '+ data[i]["date"])
    cout_journee=0    
    #pour chaque camion
    for j in range(len(data[i]["trucks"])):
        OK=True
        nb_heater, nb_clim,OK_1=Capacite_truck_respectee(data[i]["trucks"][j],j)
        OK=(OK and OK_1)        
        #camion des plants vers les warehouses      
        #vérifier que les produits de départs sont bien dans les plants
        if data[i]["trucks"][j]["home"][:1]=="P" : 
            print('le camion '+str(j)+' part d une plant')            
            OK=(OK and livraison_plant_warehouse(data[i]["trucks"][j],nb_heater,nb_clim))       
        #vérifier que les produits de départs sont bien dans la warehouse
        elif data[i]["trucks"][j]["home"][:1]=="W" :
            print('le camion '+str(j)+' part d une warehouse')
            OK=(OK and livraison_warehouse_shop(data[i]["trucks"][j],nb_heater,nb_clim))
        else:
            print('Problème:le camion ne part ni d une plant ni d une warehouse')
            OK=False       
                
        if OK:
            print('Toutes les contraintes sur le camion sont respectées, nous pouvons calculer le cout pour ce camion')
            cout_km=calcul_cout(data[i]["trucks"][j])
            print(str(cout_km)+'€')
            cout_journee+=cout_km
        else :  
            print('Certaines contraintes sur le camion ne sont pas respectées, le coût n est pas calculé !!') 
            
    #Pour chaque shop
    #Satisfaire les demandes des clients
    #Calculer les profits des ventes
    #Mettre à jour les stocks.
    local_j=0
    Shop_loc=numpy.zeros(shape=(21,4))
    for h in range(0, 21):
        Shop_loc[h,0]=Shop[h,0]
        Shop_loc[h,1]=Shop[h,1]
    for j in range(local_j,len(vente_pd)):   
        if (vente_pd.iat[j,0]==data[i]["date"]):
            for k in range(1,41):                
                vente_h=vente_pd.iat[j,k]
                #k colonne si impair heater, si pair conditionner
                partie_entiere=(k+1)//2               
                reste=(k+1)%2                   
                if Shop[partie_entiere,reste]>vente_h:
                    Shop[partie_entiere,reste]=Shop[partie_entiere,reste]-vente_h                    
                    if reste==0:
                        profit+=100*vente_h
                        Shop_loc[partie_entiere,2]=vente_h
                    else:
                        profit+=300*vente_h
                        Shop_loc[partie_entiere,3]=vente_h
                else:                   
                    if reste==0:
                        profit+=100*Shop[partie_entiere,reste]
                        cout_ventes_manquees+=100*Vente_manquee*(vente_h-Shop[partie_entiere,reste])
                        Shop_loc[partie_entiere,2]=Shop[partie_entiere,reste]
                    else :   
                        profit+=300*Shop[partie_entiere,reste]
                        cout_ventes_manquees+=300*Vente_manquee*(vente_h-Shop[partie_entiere,reste])
                        Shop_loc[partie_entiere,3]=Shop[partie_entiere,reste]
                    Shop[partie_entiere,reste]=0
            local_j=j
            break


    #Afficher_vente(Shop_loc,'Shops','Bilan de la journée '+ data[i]["date"])
    cout_global+=cout_journee
    print('cout de tranport de la journee '+str(cout_journee)+ '€')
    print('ventes de la journée '+str(profit)+ '€')
    print(' ')
argent=profit-cout_global-cout_ventes_manquees               
print ('Benefices globaux '+str(argent) )         



['2015-01-02', '2015-01-03', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19', '2015-03-20', '2015-03-21', '2015-03-23', '2015-03-24', '2015-03-25', '2015